In [1]:
import torch 
from torch import nn
import sys 
from torch.utils.data import DataLoader

sys.path.append('/home/alexabades/recsys')

from src.data.ContextDataLoader import ContextDataLoader
from src.utils.tools.tools import ROOT_PATH

In [2]:
ROOT_PATH
batch_size = 100

In [5]:
# Load preprocessed Data
from src.data.ContextDataLoader import ContextDataLoader
from src.utils.tools.tools import ROOT_PATH


train_data = ContextDataLoader(
    ROOT_PATH + "/data/processed/frappeCtxA/frappeCtxA" + ".train.rating"
)
test_data = ContextDataLoader(
    ROOT_PATH + "/data/processed/frappeCtxA/frappeCtxA" + ".test.rating"
)

# Dataloader
train_loader = DataLoader(train_data, batch_size)
test_loader = DataLoader(test_data, batch_size)

# Num User, Items Context Features
num_users = train_data.num_users
num_items = train_data.num_items
num_context = train_data.num_context


In [ ]:
from src.models.contextNFC.context_nfc import DeepNCF
from torch import nn, optim

_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [47]:
model = DeepNCF(
        num_users=num_users,
        num_items=num_items,
        num_context=num_context,
        mf_dim=8,
        layers=[31, 10, 20],
    ).to(_device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# TODO: We have to be careful, the layers minus the num of contextual features have to give a even number

for batch in train_loader:
        user_input = batch["user"].to(_device)
        item_input = batch["item"].to(_device)
        context_input = batch["context"].to(_device)
        ratings = batch["rating"].to(_device)
        # labels = labels.view(-1, 1)

        output = model(user_input, item_input, context_input)
        # loss = loss_fn(output, ratings)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        break

In [40]:
output.shape

torch.Size([100, 1])

In [48]:
import numpy as np
from typing import List

from torch import Tensor


def rmse(
    predictions: List[float | int] | Tensor, ground_truth: List[float | int] | Tensor
) -> float:
    """
    Function to calculate the RMSE from 2 lists or tensors

    Parameters:
        - predictions (List[float | int] or Tensor): Predictions made by the model
        - ground_truth (List[float | int] or Tensor): Ground truth values

    Returns:
        - RMSE (float): The root mean square error between predictions and ground truth
    """
    if len(predictions) != len(ground_truth):
        raise ValueError("Predictions and ground truth must be of the same length.")

    if isinstance(predictions, Tensor):
        if predictions.requires_grad:
            predictions = predictions.detach()
        if predictions.is_cuda:
            predictions = predictions.cpu()
        predictions = predictions.numpy()

    if isinstance(ground_truth, Tensor):
        if ground_truth.requires_grad:
            ground_truth = ground_truth.detach()
        if ground_truth.is_cuda:
            ground_truth = ground_truth.cpu()
        ground_truth = ground_truth.numpy()

    error = np.array(predictions) - np.array(ground_truth)
    squared_error = np.square(error)
    mean_squared_error = np.mean(squared_error)
    root_mean_squared_error = np.sqrt(mean_squared_error)

    return root_mean_squared_error


rmse(ratings.view(-1, 1), output)

3.851875

In [52]:
rsme_all = []
for i in range(10):
  rsme_all.append(rmse(ratings.view(-1, 1), output))

In [54]:
np.mean(rsme_all)

3.8518748

In [31]:
from torch import Tensor

type(output) == Tensor
isinstance(output, Tensor)

True

In [33]:
predictions = torch.tensor([1, 2, 3])
predictions = [1,2]

if isinstance(predictions, Tensor) and predictions.is_cuda:
  print('To CP')

In [60]:

if not 2 % 1:
  print('aa')

aa
